In [1]:
import pandas as pd
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec, KeyedVectors
from gensim import downloader
import gzip 
import math
import itertools
from time import time
from tqdm import tqdm
import tqdm.notebook as tq
import os
os.getcwd()

'G:\\My Drive\\KU\\Thesis\\Thesis'

**Format for ngram documents:**

ngram TAB year TAB match_count TAB page_count TAB volume_count NEWLINE

## First implementation

In [2]:
class NgramFiler():
    def __init__(self, coll, start_yr, end_yr):
        self.coll = coll
        self.start_yr = start_yr
        self.end_yr = end_yr
        self.decades = [str(start_yr+i*10) for i in range((end_yr-start_yr)//10)]
        
    def file(self, test=True): 
        os.chdir(f"C:\\Users\\user\\Google Drive\\KU\\Thesis\\example_files\\{self.coll}")
        write_dic = {}
        self.size_dic = {}
        for dec in self.decades:
            fname = str(dec) + ".gz"
            write_dic[dec] = gzip.open(f"../{self.coll}_processed/" + fname, "wt")
#         print(write_dic)
        if test == True:
            file_list = os.listdir()[:2]
        else:
            file_list = os.listdir()[::-1]
        for filname in tq.tqdm(file_list):
            print(filname)
            with gzip.open(filname, "rt") as fil:
                try:
                    for line in tq.tqdm(fil):
                        line_list = gensim.utils.to_unicode(line).split("\t")
                        if len(line)<3: #Why this component?
                            continue
                        # try: devolved because using string is quicker.
                        #     year = int(line[1])
                        # except ValueError:
                        #     continue
                        dec = line_list[1][:-1] + "0"
                        # dec = math.floor(year/10)*10
                        # prcssd_ngram = [word.split("_")[0] for word in ngram.lower().split()] #Get rid of POS tagging on end of words
                        # new_line = prcssed_ngram + line[1:]
                        
                        write_dic[dec].write(line)
                        # except:
                        #     print("exception!")
                        #     continue #is this what I want?
                    os.chdir("../gb_12_processed")
                    for fil in os.listdir():
                        try:
                            self.size_dic[fil].append(os.stat(fil).st_size)
                        except KeyError:
                            self.size_dic[fil] = [os.stat(fil).st_size]
                            
                    
                except OSError:
                  print("OSError: ", filname)
                  continue
                
        for dec in self.decades:
            write_dic[dec].close()
                    



#Filer = NgramFiler(coll="gb_12", start_yr=1500, end_yr=2010)
#Filer.file(test=False)

## Parallel with mgzip

In [3]:
import mgzip
class ParrNgramFiler():
    def __init__(self, coll, dire, start_yr, end_yr):
        self.coll = coll
        self.dir = dire
        self.start_yr = start_yr
        self.end_yr = end_yr
        self.decades = [str(start_yr+i*10) for i in range((end_yr-start_yr)//10)]
        
    def file(self, test=True): 
        os.chdir(f"{self.dir}/{self.coll}")
        write_dic = {}
        self.size_dic = {}
        for dec in self.decades:
            fname = str(dec) + ".gz"
            write_dic[dec] = mgzip.open(f"../{self.coll}_processed/" + fname, "wt", encoding="utf-8")
#         print(write_dic)
        if test == True:
            file_list = os.listdir()[:2]
        else:
            file_list = os.listdir()[::-1]
        for filname in tq.tqdm(file_list):
            print(filname)
            with mgzip.open(f"{self.dir}/{self.coll}/" + filname, "rt", encoding="utf-8") as fil:
                try:
                    for line in tq.tqdm(fil):
                        line_list = gensim.utils.to_unicode(line).split("\t")
                        if len(line)<3: #Why this component?
                            continue
                        # try: devolved because using string is quicker.
                        #     year = int(line[1])
                        try:
                            if int(line[2]) < 40:
                                continue
                        except:
                            pass
                        # except ValueError:
                        #     continue
                        dec = line_list[1][:-1] + "0"
                        # dec = math.floor(year/10)*10
                        # prcssd_ngram = [word.split("_")[0] for word in ngram.lower().split()] #Get rid of POS tagging on end of words
                        # new_line = prcssed_ngram + line[1:]
                        
                        write_dic[dec].write(line)
                        # except:
                        #     print("exception!")
                        #     continue #is this what I want?
                    os.chdir("../gb_12_processed")
                    for fil in os.listdir():
                        try:
                            self.size_dic[fil].append(os.stat(fil).st_size)
                        except KeyError:
                            self.size_dic[fil] = [os.stat(fil).st_size]
                            
                    
                except OSError:
                  print("OSError: ", filname)
                  continue
                
        for dec in self.decades:
            write_dic[dec].close()
                    



#Filer = ParrNgramFiler(coll="eng_fic_12", dire="D:/google_ngrams", start_yr=1500, end_yr=2010)
#Filer.file(test=False)

# Partitioning and deleting entries under 40 matches

In [4]:
#Remove folder
import shutil

#shutil.rmtree("D:/google_ngrams/eng_fic_12_processed_2")

In [ ]:
import mgzip
import string
from datetime import datetime
from time import time

class ParrNgramFiler():
    def __init__(self, coll, out, dire, start_yr, end_yr, matchmin=25):
        self.coll = coll
        self.out = out
        self.dir = dire
        self.start_yr = start_yr
        self.end_yr = end_yr
        self.decades = [start_yr+i*10 for i in range((end_yr-start_yr)//10)]
        self.matchmin = matchmin
        self.exclist = string.punctuation + string.digits
        self.punc = string.punctuation
        
    def file(self, test=True): 
        os.chdir(f"{self.dir}/{self.coll}")
        write_dic = {}
        self.size_dic = {}
        if test:
            try:
                shutil.rmtree(f"../{self.coll}_{self.out}/")
            except:
                pass
        os.makedirs(f"../{self.coll}_{self.out}/")
        for dec in self.decades:
            fname = str(dec) + ".gz"
            write_dic[dec] = mgzip.open(f"../{self.coll}_{self.out}/" + fname, "wt", encoding="utf-8")
#         print(write_dic)
        if test != 0:
            file_list = os.listdir()[:test:-1]
        else:
            file_list = os.listdir()
        for filname in tq.tqdm(file_list):
            print("File: ", filname, f", current time: {time()-start}")
            with mgzip.open(f"{self.dir}/{self.coll}/" + filname, "rt", encoding="utf-8") as fil:
                try:
                    for i, line in enumerate(tq.tqdm(fil)):
                        line = gensim.utils.to_unicode(line).split("\t")
                        #if len(line)<3:
                         #   print("fail")
                         #   continue
                            
                        try:
                            year = int(line[1])
                        except ValueError:
                            continue
                        dec = math.floor(year/10)*10
                        
                        ngram = line[0]
                        #print(ngram)
                        if i == 0:
                            prev_ngram = ngram
                            match_count = int(line[2])
                            current_dec = dec
                            continue
                        try: 
                            matches = int(line[2])
                        except Exception as e:
                            print(f"Error, line {i} -> {line}:\n", e)
                            continue
                       # print(ngram, matches, int(line[2]), prev_ngram)
                        #if i < 1000:
                           #print(line, "\n", ngram, "\n", prev_ngram, "\n", matches, current_dec, "\n\n")
                        if prev_ngram == ngram:
                            if dec == current_dec:
                                match_count += matches
                            else: 
                                if match_count > self.matchmin:
                                    #table_ = str.maketrans('', '', self.exclist)
                                    #newngram = ngram.translate(table_)
                                    #split = [word.split("_")[0] for word in newngram.lower().split()]
                                    split = []
                                    for word in ngram.lower().split():
                                        word = word.split("_")[0]
                                        if word in self.punc:
                                            continue
                                        else:
                                            split.append(word)
                                    if len(split) > 3:
                                        to_write = " ".join(split + ["\t" + str(match_count) + "\n"]) #Get rid of POS tagging on end of words
                                        #print("TO WRITE: ", to_write)
                                        write_dic[current_dec].write(to_write)
                                    current_dec = dec
                                else:
                                    current_dec = dec
                                    match_count = matches
                                
                        else:
                            if match_count > self.matchmin:
                                split = []
                                for word in ngram.lower().split():
                                    word = word.split("_")[0]
                                    if word in self.punc:
                                        continue
                                    else:
                                        split.append(word)
                                
                                if len(split) > 3:
                                    to_write = " ".join(split + ["\t" + str(match_count) + "\n"]) #Get rid of POS tagging on end of words
                                    write_dic[current_dec].write(to_write)
                                    #print("\nTO WRITE: ",to_write)
                            prev_ngram = ngram
                            match_count = matches
                            current_dec = dec
                         
                                                        
                    
                except OSError:
                    print("OSError: ", filname)
                    continue
                
        for dec in self.decades:
            write_dic[dec].close()
                    


start = time()
Filer = ParrNgramFiler(coll="us_12", out="filtered_after_error3", dire="D:/google_ngrams", start_yr=1500, end_yr=2010)
Filer.file(test=False)
print(f"Total time taken: {time()-start}")

  0%|          | 0/722 [00:00<?, ?it/s]

File:  a_ , current time: 0.304187536239624


0it [00:00, ?it/s]

File:  aa , current time: 1924.9089839458466


0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

File:  mn , current time: 75790.83288860321


0it [00:00, ?it/s]

File:  mo , current time: 75808.03115916252


0it [00:00, ?it/s]

## Check whether each file iterated over writes more information to the output files

In [ ]:
NgramFiler.size_dic



In [ ]:
start = time()
Filer = ParrNgramFiler(coll="gb_12", out="filtered_full", dire="D:/google_ngrams", start_yr=1500, end_yr=2010)
Filer.file(test=False)
print(f"Total time taken: {time()-start}")